# Task 3 – Exploratory Analysis (Python)

## 1. Data Understanding & Cleaning
- Load the datasets
- Handle missing values or outliers if necessary

In [58]:
# Your code here
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# Load data
df_events = pd.read_csv('../data/events.csv')
df_devices = pd.read_csv('../data/devices.csv')
df_users = pd.read_csv('../data/users.csv')
# number of rows and columns.
print("Users shape:", df_users.shape)
# Print first 5 rows to understand the data structure
print(df_users.head(),"\n")

# statistical summary of the dataset
print(df_users.describe(),"\n")

print("Missing values (Users):")
print(df_users.isnull().sum())
# Unique values for categorical columns
print("\nUnique Regions:")
print(df_users["region"].unique())

print("\nUnique Platforms:")
print(df_users["platform"].unique())


Users shape: (121, 4)
    user_id signup_date         region  platform
0  739b720a  2023-04-21  United States       iOS
1  0aa3325c  2023-06-12             US       iOS
2  b5a2048e  2023-08-20             KR       Web
3  a4a70166  2023-05-26            can  3rdParty
4  2386e2a0  2023-01-18            USA   Android 

         user_id signup_date         region platform
count        121         121            121      121
unique       120         103              8        4
top     739b720a  2023-04-21  United States      iOS
freq           2           2             18       32 

Missing values (Users):
user_id        0
signup_date    0
region         0
platform       0
dtype: int64

Unique Regions:
['United States' 'US' 'KR' 'can' 'USA' 'CA' 'JP' 'Canada']

Unique Platforms:
['iOS' 'Web' '3rdParty' 'Android']


In [59]:
# number of rows and columns.
print("devices shape:", df_devices.shape)
# Print first 5 rows to understand the data structure
print(df_devices.head(), "\n")

# statistical summary of the dataset
print(df_devices.describe(), "\n")

print("Missing values (Devices):")
print(df_devices.isnull().sum())
# Unique values for categorical columns

print("\nUnique Networks:")
print(df_devices["network"].unique())

print("\nUnique Device Types:")
print(df_devices["device_type"].unique())

print("\nUnique Firmware Versions:")
print(df_devices["firmware_version"].unique())

print("\nUnique Locations:")
print(df_devices["location"].unique())
# Clean missing location to Unknown and
df_devices["location"] = df_devices["location"].fillna("Unknown")
df_devices["user_id_missing"] = df_devices["user_id"].isnull()
print(f"Devices location nulls filled: {(df_devices['location'] == 'Unknown').sum()}")
print(f"Devices with no user_id flagged: {df_devices['user_id_missing'].sum()}")
print(f"Devices on beta firmware: {(df_devices['firmware_version'] == '3.5.beta').sum()}")

devices shape: (450, 6)
    device_id   user_id network device_type firmware_version location
0  d_f4abcb9e  89a25ef6    tuya         fan            1.0.3   Office
1  d_212b804b  2f555493    tuya  smart_plug            1.0.0      NaN
2  d_32b272b3  f41317b2    tuya      heater         3.5.beta      NaN
3  d_f6fe781d  ee413bc3    tuya         fan         3.5.beta      NaN
4  d_8555efd8  cd6e76a2    tuya  smart_bulb         3.5.beta     Home 

         device_id   user_id network device_type firmware_version location
count          450       425     450         450              450      342
unique         450       119       2           8                4        3
top     d_9f01e3d6  739b720a    tuya  thermostat            2.1.0     Home
freq             1        10     228          70              115      136 

Missing values (Devices):
device_id             0
user_id              25
network               0
device_type           0
firmware_version      0
location            108
dtype: 

In [60]:
# number of rows and columns.
print("events shape:", df_events.shape)
# Print first 5 rows to understand the data structure
print(df_events.head(), "\n")
# summary
print(df_events.describe(), "\n")

print("Missing values (Events):")
print(df_events.isnull().sum())

import json
# get two raw payloads with different structures
sample_metadata = df_events[df_events["payload"].str.contains('"metadata"')].iloc[0]
sample_status = df_events[df_events["payload"].str.contains('"status"')].iloc[0]

print("── Payload A ──")
print(json.dumps(json.loads(sample_metadata["payload"]), indent=2))
print("── Payload B ──")
print(json.dumps(json.loads(sample_status["payload"]), indent=2))
# See what network these devices actually belong to
device_a = df_devices[df_devices["device_id"] == sample_metadata["device_id"]][
    ["device_id", "network"]
]
device_b = df_devices[df_devices["device_id"] == sample_status["device_id"]][
    ["device_id", "network"]
]

print("\nDevice A network:", device_a["network"].values[0])
print("Device B network:", device_b["network"].values[0])



events shape: (15000, 6)
  event_id   device_id event_type  event_value                  event_ts  \
0      e_0  d_87cfe321  telemetry  see_payload  2024-01-18T17:45:43.000Z   
1      e_1  d_fddd9d33  telemetry  see_payload  2024-02-04T16:18:19.000Z   
2      e_2  d_c5859fb6  telemetry  see_payload  2024-01-12T10:57:04.000Z   
3      e_3  d_de3f2763  telemetry  see_payload  2024-01-02T08:09:18.000Z   
4      e_4  d_8406c179  telemetry  see_payload  2024-01-23T01:15:16.000Z   

                                             payload  
0  {"metadata": {"oem_model": "door_sensor", "dsn...  
1  {"metadata": {"oem_model": "window_sensor", "d...  
2  {"status": [{"code": "generic_state", "value":...  
3  {"status": [{"code": "switch_led", "value": fa...  
4  {"status": [{"code": "generic_state", "value":...   

       event_id   device_id event_type  event_value                  event_ts  \
count     15000       15000      15000        15000                     15000   
unique    15000         

In [61]:
# Cleaning users.csv
# only US and Canada have the messy variants. Any region not in region_map will be set to Other
region_map = {
    "United States": "US",
    "USA": "US",
    "US": "US",
    "Canada": "CA",
    "can": "CA",
    "CA": "CA",
    "KR": "KR",
    "JP": "JP",
}
df_users["region"] = df_users["region"].map(region_map).fillna("Other")

# Drop duplicate user_id
dupes = df_users["user_id"].duplicated().sum()
print(f"\nDuplicate user_ids dropped: {dupes}")
df_users = df_users.drop_duplicates(subset="user_id", keep="first")

# Parse signup_date to datetime
df_users["signup_date"] = pd.to_datetime(df_users["signup_date"])

print("Users cleaned:", df_users.shape)
print("Region counts after normalization:")
print(df_users["region"].value_counts().to_string(index=True))


Duplicate user_ids dropped: 1
Users cleaned: (120, 4)
Region counts after normalization:
region
US    47
CA    45
KR    17
JP    11


In [62]:
# event_value is always "see_payload" real data in payload


def parse_payload(raw):
    try:
        data = json.loads(raw)
    except (json.JSONDecodeError, TypeError):
        return pd.Series({
            'source':           'parse_error',
            'extracted_code':   None,
            'extracted_value':  None,
            'voltage':          None,
            'current':          None,
            'power':            None,
            'temperature_c':    None,
        })

    # ── Ayla structure ──
    if 'metadata' in data and 'datapoint' in data:
        prop  = data['datapoint'].get('property')
        value = data['datapoint'].get('value')
        return pd.Series({
            'source':           'ayla',
            'extracted_code':   prop,
            'extracted_value':  value,
            'voltage':          None,
            'current':          None,
            'power':            None,
            'temperature_c':    value if prop == 'local_temperature' else None,
        })

    # ── Tuya structure ──
    if 'status' in data and isinstance(data['status'], list) and data['status']:
        # index all codes in this event
        status_map = {item['code']: item['value'] for item in data['status']}

        # primary code is still the first one (for extracted_code/value)
        first = data['status'][0]

        return pd.Series({
            'source':           'tuya',
            'extracted_code':   first.get('code'),
            'extracted_value':  first.get('value'),
            # dedicated columns for specific measurements
            'voltage':          status_map.get('cur_voltage'),   # raw: divide by 10 for volts
            'current':          status_map.get('cur_current'),   # raw: divide by 1000 for amps
            'power':            status_map.get('cur_power'),     # raw: divide by 10 for watts
            'temperature_c':    status_map.get('local_temperature'),
        })

    return pd.Series({
        'source':           'unknown',
        'extracted_code':   None,
        'extracted_value':  None,
        'voltage':          None,
        'current':          None,
        'power':            None,
        'temperature_c':    None,
    })


print("\nParsing payload column (15,000 rows)...")
parsed    = df_events['payload'].apply(parse_payload)
df_events = pd.concat([df_events, parsed], axis=1)

# Parse timestamp
df_events['event_ts'] = pd.to_datetime(df_events['event_ts'], utc=True)
df_events['date']     = df_events['event_ts'].dt.date

# Convert extracted_value to numeric when possible
# Booleans (True/False) become 1/0 — meaningful for switch/sensor events
df_events['extracted_value_numeric'] = pd.to_numeric(
    df_events['extracted_value'].apply(
        lambda x: int(x) if isinstance(x, bool) else x
    ),
    errors='coerce'
)

# Scale raw Tuya electrical values to proper units
df_events['voltage_v'] = df_events['voltage'] / 10
df_events['current_a'] = df_events['current'] / 1000
df_events['power_w']   = df_events['power']   / 10

print("✓ Events cleaned:", df_events.shape)
print("\nPayload source breakdown:")
print(df_events['source'].value_counts().to_string())
print("\nTop extracted codes:")
print(df_events['extracted_code'].value_counts().head(10).to_string())
print("\nAll unique extracted codes:")
print(df_events["extracted_code"].value_counts().to_string())
print("\nDedicated measurement columns:")
print(f"  voltage_v readings:  {df_events['voltage_v'].notna().sum()}")
print(f"  current_a readings:  {df_events['current_a'].notna().sum()}")
print(f"  power_w readings:    {df_events['power_w'].notna().sum()}")
print(f"  temperature_c readings: {df_events['temperature_c'].notna().sum()}")

# verify devices and network
df_check = df_events.merge(
    df_devices[["device_id", "network"]], on="device_id", how="left"
)
print(df_check.groupby(["network", "source"]).size())

# FINAL SUMMARY
print("\n─── Final Dataset Summary ───")
print(f"Users:   {df_users.shape[0]} rows | Regions: {sorted(df_users['region'].unique())}")
print(f"Devices: {df_devices.shape[0]} rows | Networks: {df_devices['network'].unique().tolist()}")
print(f"Events:  {df_events.shape[0]} rows | Date range: {df_events['date'].min()} → {df_events['date'].max()}")


Parsing payload column (15,000 rows)...
✓ Events cleaned: (15000, 18)

Payload source breakdown:
source
tuya    8319
ayla    6681

Top extracted codes:
extracted_code
generic_state        4999
contact_state        3228
local_temperature    2092
switch_led           1890
switch_1             1430
connectivity         1361

All unique extracted codes:
extracted_code
generic_state        4999
contact_state        3228
local_temperature    2092
switch_led           1890
switch_1             1430
connectivity         1361

Dedicated measurement columns:
  voltage_v readings:  1430
  current_a readings:  1430
  power_w readings:    1430
  temperature_c readings: 2092
network  source
ayla     ayla      6681
tuya     tuya      8319
dtype: int64

─── Final Dataset Summary ───
Users:   120 rows | Regions: ['CA', 'JP', 'KR', 'US']
Devices: 450 rows | Networks: ['tuya', 'ayla']
Events:  15000 rows | Date range: 2024-01-01 → 2024-02-07


## 2. Visualizations
Produce at least 3 meaningful charts, such as:
- Event volume over time
- Events per device or per user
- Comparison between Ayla vs Tuya devices
- Highlight any anomalies or interesting patterns

In [63]:
# Your code here

# Task 4 – Hypotheses & Questions

1. Propose 2–3 hypotheses about user or device behavior.
2. Show how you would test each hypothesis with the available data.
3. Clearly state whether the data supports, partially supports, or does not support the hypothesis.

In [64]:
# Your code for hypothesis testing here

# Task 5 – Reflection

- What additional data would improve this analysis?
- What limitations prevent deeper insights?
- What would you explore next if this were Phase 0 of a larger project?

*Write your reflection here...*